In [7]:
# Using Google Generative AI directly (no LlamaIndex needed)
import google.generativeai as genai

In [8]:
import setup
setup.init_django()

In [10]:
from decouple import config
from blog.models import BlogPost, EMBEDDING_LENGTH
from blog import services
import google.generativeai as genai

In [11]:
qs = BlogPost.objects.filter(can_delete=True)

In [ ]:
LLM_MODEL = config('LLM_MODEL', default='gemini-1.5-flash')
EMBEDDING_LENGTH = config("EMBEDDING_LENGTH", default=768, cast=int)
EMBEDDING_MODEL = config("EMBEDDING_MODEL", default='embedding-001')
GEMINI_API_KEY = config("GEMINI_API_KEY")

In [13]:
# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Create the model
model = genai.GenerativeModel(LLM_MODEL)
print(f"Using model: {LLM_MODEL}")

C:\Users\raghu\AppData\Local\Temp\ipykernel_14600\2678343577.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(


In [14]:
# Test the model
response = model.generate_content("Hello! Can you tell me about yourself?")
print(response.text)

In [15]:
# Simple query function using Gemini directly
def query_blog_posts(question):
    # Get blog posts
    posts = BlogPost.objects.all()[:5]
    
    # Create context
    context = "\n\n".join([f"Post: {post.title}\n{post.content[:300]}..." for post in posts])
    
    # Generate response
    prompt = f"Based on these blog posts, answer: {question}\n\nPosts:\n{context}"
    response = model.generate_content(prompt)
    return response.text

# Test it
answer = query_blog_posts("What are the main topics?")
print(answer)

In [16]:
vector_db_name = "vector_db"
vector_db_table_name = "blogpost"

In [17]:
DATABASE_URL = config("DATABASE_URL_POOL")
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

In [18]:
# create a new Database:
from sqlalchemy import create_engine, text

engine = create_engine(DATABASE_URL, isolation_level='AUTOCOMMIT')
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1 FROM pg_database WHERE datname = :db_name"), {"db_name":vector_db_name})
    db_exists = result.scalar() == 1
    
    if not db_exists:
        # Create the database
        connection.execute(text(f"CREATE DATABASE {vector_db_name}"))
        print(f"Created database: {vector_db_name}")
    else:
        print(f"Database {vector_db_name} already exists")

Database vector_db already exists


In [19]:
# Blog post analysis with Gemini
if qs.exists():
    # Get the first few posts
    posts = qs[:3]
    
    # Create a prompt with the posts
    prompt = f"""
    Based on these blog posts, provide a summary of the main topics discussed:
    
    {chr(10).join([f'Post {i+1}: {post.title}\n{post.content[:300]}...' for i, post in enumerate(posts)])}
    
    Please provide a concise summary of the main topics and themes.
    """
    
    response = model.generate_content(prompt)
    print("Blog Analysis:")
    print(response.text)
else:
    print("No blog posts found with can_delete=True")

In [ ]:
# Combine search with Gemini for Q&A
def ask_about_blog_posts(question):
    """
    Search for relevant blog posts and use Gemini to answer questions about them
    """
    # Search for relevant posts
    search_results = services.search_posts(question, limit=3)
    
    if not search_results:
        return "No relevant blog posts found."
    
    # Create context from search results
    context = "\n\n".join([
        f"Post: {result.title}\n{result.content[:500]}..." 
        for result in search_results
    ])
    
    # Create prompt
    prompt = f"""
    Based on the following blog posts, answer this question: {question}
    
    Blog posts:
    {context}
    
    Please provide a comprehensive answer based on the content above.
    """
    
    response = model.generate_content(prompt)
    return response.text

# Test the Q&A function
if qs.exists():
    question = "What are the main topics discussed in the blog posts?"
    answer = ask_about_blog_posts(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")

In [ ]:
# Simple query function using Gemini directly
def query_blog_posts(question):
    # Get blog posts
    posts = BlogPost.objects.all()[:5]
    
    # Create context
    context = "\n\n".join([f"Post: {post.title}\n{post.content[:300]}..." for post in posts])
    
    # Generate response
    prompt = f"Based on these blog posts, answer: {question}\n\nPosts:\n{context}"
    response = model.generate_content(prompt)
    return response.text

# Test it
answer = query_blog_posts("What are the main topics?")
print(answer)